# Lab3 - Second part: Machine learning in Spark
## Challenge - Linear Classification Methods

Repeat this task, trying out at least two additional linear classification methods. For example you could try L1 regularized SVM, or to run SVM without regularization. Another alternative is to run Logistic Regression, with L1 or L2 regularization. Report your findings in terms of model performance.